In [1]:
import pandas as pd
from snp_query_box import DsnpHelperFunction, snpQueries
import datetime as dt

today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')

last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
last_date_of_last_month_str = last_date_of_last_month.strftime("%Y-%m-%d")

last_month = last_date_of_last_month_str[:7]

In [2]:
user_id = "A845514"


pulled_data_date = last_month
monthly_pull_data_path = f"C:/Users/{user_id}/Projects_Code/data/monthly_data_dump/{pulled_data_date}/pull_db"

pull_ip_visit = pd.read_pickle(f"{monthly_pull_data_path}/pull_ip_visit_12month.pkl")


In [3]:
pull_ip_visit

,SRC_MEMBER_ID,CMS_CNTRCT_NBR,group,PHONE_NBR,PROVIDER_ID,GROUP,GROUP_NAME,SRV_START_DT,SRV_STOP_DT
0,101039822500,H5521,1,804-225-8697,740432909,453145533,International Natura Clinic Center LLC,2024-01-28,2024-01-31
1,101039825900,H5521,2,703-300-0602,712070409,305865000,Signature Partners Network Medicare,2023-08-17,2023-08-23
2,101039825900,H5521,3,703-300-0602,712070409,305865000,Signature Partners Network Medicare,2023-08-30,2023-09-03
3,101039825900,H5521,4,703-300-0602,712070409,305865000,Signature Partners Network Medicare,2023-09-19,2023-09-21
4,101039825900,H5521,5,703-300-0602,712070409,305865000,Signature Partners Network Medicare,2023-11-26,2023-12-01
...,...,...,...,...,...,...,...,...,...
696655,101947845200,H3931,696656,443-572-2172,517747309,546317810,LifeBridge Community Physicians Inc,2024-02-13,2024-02-15
696656,101947845200,H3931,696657,443-572-2172,517747309,546317810,LifeBridge Community Physicians Inc,2024-02-20,2024-02-26
696657,101949279200,H5521,696658,509-551-5176,9,None,Unknown,2024-02-19,2024-02-22
696658,101950155500,H5793,696659,203-888-9811,427673209,681187510,Northeast Medical Group State of CT,2024-02-18,2024-02-20


In [4]:
pull_ip_visit.groupby("SRC_MEMBER_ID").size().sort_values()

SRC_MEMBER_ID
101039822500      1
101453392800      1
101453391800      1
101453386000      1
101453382600      1
               ... 
101547531500     48
101552697600     51
101591039600     52
101312384300     63
101515500800    109
Length: 421510, dtype: int64

In [56]:
test_df = pull_ip_visit[pull_ip_visit["SRC_MEMBER_ID"].isin(["101552697600", "101591039600"])]
len(test_df)

103

In [61]:
test_df.SRC_MEMBER_ID.value_counts()

SRC_MEMBER_ID
101591039600    52
101552697600    51
Name: count, dtype: int64

In [113]:
def _collapse_intervals(intervals):
    intervals.sort(key=lambda x: x[0])
    collapsed = []
    for start, end in intervals:
        if not collapsed or collapsed[-1][1] < start:
            collapsed.append([start, end])
        else:
            collapsed[-1][1] = max(collapsed[-1][1], end)
    return collapsed


def collapse_service_date_visit(pull_df):
    df = pull_df[["SRC_MEMBER_ID", "SRV_START_DT", "SRV_STOP_DT", "CMS_CNTRCT_NBR", "PHONE_NBR", "PROVIDER_ID", "GROUP", "GROUP_NAME"]]
    df = df.sort_values(by=["SRC_MEMBER_ID", "SRV_START_DT"]).reset_index(drop=True)
    
    collapsed_records = []
    current_id = df["SRC_MEMBER_ID"].iloc[0]
    current_intervals = [(df['SRV_START_DT'].iloc[0], df['SRV_STOP_DT'].iloc[0])]
    for i in range(1, len(df)):
        if df['SRC_MEMBER_ID'].iloc[i] == current_id:
            current_intervals.append((df['SRV_START_DT'].iloc[i], df['SRV_STOP_DT'].iloc[i]))
        else:
            #when id change, add to the result
            collapsed_records.extend([(current_id, start, end) for start, end in _collapse_intervals(current_intervals)])
            current_id = df['SRC_MEMBER_ID'].iloc[i]
            current_intervals = [(df['SRV_START_DT'].iloc[i], df['SRV_STOP_DT'].iloc[i])]
    collapsed_records.extend([(current_id, start, end) for start, end in _collapse_intervals(current_intervals)])
    collapsed_df = pd.DataFrame(collapsed_records, columns = ['SRC_MEMBER_ID', 'SRV_START_DT', 'SRV_STOP_DT'])


    result_df = collapsed_df.merge(df.drop(columns = "SRV_STOP_DT"), on = ["SRC_MEMBER_ID","SRV_START_DT"], how="left")
    result_df = result_df.drop_duplicates(keep = 'first')
    
    return result_df

In [114]:
new_df = collapse_service_date_visit(pull_ip_visit)

In [115]:
new_df

,SRC_MEMBER_ID,SRV_START_DT,SRV_STOP_DT,CMS_CNTRCT_NBR,PHONE_NBR,PROVIDER_ID,GROUP,GROUP_NAME
0,101039822500,2024-01-28,2024-01-31,H5521,804-225-8697,740432909,453145533,International Natura Clinic Center LLC
1,101039825900,2023-08-17,2023-08-23,H5521,703-300-0602,712070409,305865000,Signature Partners Network Medicare
2,101039825900,2023-08-30,2023-09-03,H5521,703-300-0602,712070409,305865000,Signature Partners Network Medicare
3,101039825900,2023-09-19,2023-09-21,H5521,703-300-0602,712070409,305865000,Signature Partners Network Medicare
4,101039825900,2023-11-26,2023-12-01,H5521,703-300-0602,712070409,305865000,Signature Partners Network Medicare
...,...,...,...,...,...,...,...,...
672466,101947845200,2024-02-13,2024-02-15,H3931,443-572-2172,517747309,546317810,LifeBridge Community Physicians Inc
672467,101947845200,2024-02-20,2024-02-26,H3931,443-572-2172,517747309,546317810,LifeBridge Community Physicians Inc
672468,101949279200,2024-02-19,2024-02-22,H5521,509-551-5176,9,None,Unknown
672469,101950155500,2024-02-18,2024-02-20,H5793,203-888-9811,427673209,681187510,Northeast Medical Group State of CT


In [116]:
pull_ip_visit.SRC_MEMBER_ID.value_counts().reset_index().head(10)

,SRC_MEMBER_ID,count
0,101515500800,109
1,101312384300,63
2,101591039600,52
3,101552697600,51
4,101547531500,48
5,101186854100,47
6,101046385900,46
7,101594430700,45
8,101694208200,44
9,101150990500,43


In [117]:
new_df.SRC_MEMBER_ID.value_counts().reset_index().head(10)

,SRC_MEMBER_ID,count
0,101515500800,109
1,101312384300,63
2,101591039600,52
3,101186854100,47
4,101594430700,45
5,101694208200,44
6,101046385900,43
7,101150990500,43
8,101220986100,42
9,101547531500,39


In [118]:
# 2023-09-29	2023-11-12 not collapsed
pull_ip_visit[pull_ip_visit["SRC_MEMBER_ID"] =="101547531500"]

,SRC_MEMBER_ID,CMS_CNTRCT_NBR,group,PHONE_NBR,PROVIDER_ID,GROUP,GROUP_NAME,SRV_START_DT,SRV_STOP_DT
513550,101547531500,H3312,513551,929-303-2143,917925609,435962170,ALEDADE_NY,2023-04-01,2023-04-02
513551,101547531500,H3312,513552,929-303-2143,917925609,435962170,ALEDADE_NY,2023-04-08,2023-04-21
513552,101547531500,H3312,513553,929-303-2143,917925609,435962170,ALEDADE_NY,2023-04-23,2023-04-26
513553,101547531500,H3312,513554,929-303-2143,917925609,435962170,ALEDADE_NY,2023-04-28,2023-04-30
513554,101547531500,H3312,513555,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-06,2023-05-07
513555,101547531500,H3312,513556,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-12,2023-05-13
513556,101547531500,H3312,513557,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-15,2023-05-15
513557,101547531500,H3312,513558,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-17,2023-05-17
513558,101547531500,H3312,513559,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-19,2023-05-21
513559,101547531500,H3312,513560,929-303-2143,917925609,435962170,ALEDADE_NY,2023-05-24,2023-05-25


In [119]:
new_df[new_df["SRC_MEMBER_ID"] =="101547531500"]

,SRC_MEMBER_ID,SRV_START_DT,SRV_STOP_DT,CMS_CNTRCT_NBR,PHONE_NBR,PROVIDER_ID,GROUP,GROUP_NAME
495892,101547531500,2023-04-01,2023-04-02,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495893,101547531500,2023-04-08,2023-04-21,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495894,101547531500,2023-04-23,2023-04-26,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495895,101547531500,2023-04-28,2023-04-30,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495896,101547531500,2023-05-06,2023-05-07,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495897,101547531500,2023-05-12,2023-05-13,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495898,101547531500,2023-05-15,2023-05-15,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495899,101547531500,2023-05-17,2023-05-17,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495900,101547531500,2023-05-19,2023-05-21,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
495901,101547531500,2023-05-24,2023-05-25,H3312,929-303-2143,917925609,435962170,ALEDADE_NY
